## FDSN Waveform Downloader, Converter & Detector

Downloads I06AU and I52GB waveforms from IRIS FSDN services using Obpsy Mass Downloader and then converts them the .SAC file with it's stats for further processing. The .SAC files will be then further processed using "Bartlett" beamforming method and Adaptive F-Detector to identify signals most related to Anak Krakatau Volcanic Activity. These specific volcanic activity signals will be later ingested into the algorithm for the Deep Learning processing

This notebook consists of 3 parts:
1. Dowloading Data using Obspy Mass Downloader
2. Conversion of MSEED Files to .SAC
3. Conversion of .SAC files to CSV
4. Implementing FK Beamforming to each station group
5. Running Adaptive F-Detector to Isolate relevant Waveform

In [61]:
import sys
sys.path.append('/run/media/viblab/Markov11/Haykal/AnakKrakatauEWS/')

import obspy
from obspy import UTCDateTime, read, read_inventory
from obspy.clients.fdsn.mass_downloader import GlobalDomain, \
    Restrictions, MassDownloader
from obspy.core.util.attribdict import AttribDict
import os
import subprocess
import glob
from tqdm.notebook import tqdm
from src.utils import *

### 1. Downloading Data Using Obspy Mass Downloader

Download the data ranging from 2018-06-24T00:00:00 until 2019-09-03T00:00:00

In [ ]:
# 1.1 I06AU Waveform Download

domain = GlobalDomain()

restrictions = Restrictions(
    # Get data for a whole year.
    starttime=obspy.UTCDateTime(2018, 6, 24),
    endtime=obspy.UTCDateTime(2019, 9, 3),
    # Chunk it to have one file per day.
    chunklength_in_sec=86400,
    network="IM", station="I06H*", location="", channel="BDF",
    # The typical use case for such a data set are noise correlations where
    # gaps are dealt with at a later stage.
    reject_channels_with_gaps=False,
    # Same is true with the minimum length. All data might be useful.
    minimum_length=0.0,
    # Guard against the same station having different names.
    minimum_interstation_distance_in_m=100.0)

mdl = MassDownloader(providers=["IRIS"])
mdl.download(domain, restrictions, mseed_storage="waveform_collection/I06AU/WAVEFORM_I06AU_MSEED",
             stationxml_storage="waveform_collection/I06AU/I06AU_STATIONS")

In [ ]:
# 1.2 I52GB Waveform Download

domain = GlobalDomain()

restrictions = Restrictions(
    starttime=obspy.UTCDateTime(2018, 6, 24),
    endtime=obspy.UTCDateTime(2019, 9, 3),
    chunklength_in_sec=86400,
    network="IM", station="I52H*", location="", channel="BDF",
    reject_channels_with_gaps=False,
    minimum_length=0.0,
    minimum_interstation_distance_in_m=100.0)


mdl = MassDownloader(providers=["IRIS"])
mdl.download(domain, restrictions, mseed_storage="waveform_collection/I52GB/WAVEFORM_I52GB_MSEED",
             stationxml_storage="waveform_collection/I52GB/I52GB_STATIONS")

### 2. Conversion of MSEED Files to .SAC
Converting MSEED Files to SAC Complete with Important Headers

In [ ]:
# 2.1 I06AU Waveform Conversion

input_directory = 'waveform_collection/I06AU/WAVEFORM_I06AU_MSEED'
output_directory = 'waveform_collection/I06AU/WAVEFORM_I06AU_SAC'
stationxml_directory = 'waveform_collection/I06AU/I06AU_STATIONS'

# Run the Function
mseed_to_sac(input_directory, output_directory, stationxml_directory)

In [ ]:
# 2.2 I52GB Waveform Conversion

input_directory = 'waveform_collection/I52GB/WAVEFORM_I52GB_MSEED'
output_directory = 'waveform_collection/I52GB/WAVEFORM_I52GB_SAC'
stationxml_directory = 'waveform_collection/I52GB/I52GB_STATIONS'

# Run the Function
mseed_to_sac(input_directory, output_directory, stationxml_directory)


### 3. Conversion of SAC files to CSV

In [60]:
# 3.1 I06AU SAC to csv conversion

folder_path = '/run/media/viblab/Markov11/Haykal/AnakKrakatauEWS/waveform_collection/I06AU/WAVEFORM_I06AU_SAC'
output_folder = '/run/media/viblab/Markov11/Haykal/AnakKrakatauEWS/waveform_collection/I06AU/WAVEFORM_I06AU_CSV'
freqmin = 0.7 
freqmax = 4.0 
df_grouped_file_paths = sac_path_grouping(folder_path)

# Iterate over the DataFrame rows and process each file path
for index, row in df_grouped_file_paths.iterrows():
    file_path_pattern = row['GroupedFilePath']
    try:
        # Process each file path pattern with the sac_to_csv function
        csv_output_path = sac_to_csv(file_path_pattern, output_folder, freqmin, freqmax)
        print(f'CSV file created at: {csv_output_path}')
    except Exception as e:
        print(f"Error processing file pattern {file_path_pattern}: {e}")

<class 'pandas.core.frame.DataFrame'>
CSV file created at: /run/media/viblab/Markov11/Haykal/AnakKrakatauEWS/waveform_collection/I06AU/WAVEFORM_I06AU_CSV/IM.I06H_..BDF__20180625T000000Z__20180626T000000Z.csv
CSV file created at: /run/media/viblab/Markov11/Haykal/AnakKrakatauEWS/waveform_collection/I06AU/WAVEFORM_I06AU_CSV/IM.I06H_..BDF__20180626T000000Z__20180627T000000Z.csv
CSV file created at: /run/media/viblab/Markov11/Haykal/AnakKrakatauEWS/waveform_collection/I06AU/WAVEFORM_I06AU_CSV/IM.I06H_..BDF__20180624T000000Z__20180625T000000Z.csv


In [ ]:
# 3.1 I52GB SAC to csv conversion

folder_path = '/run/media/viblab/Markov11/Haykal/AnakKrakatauEWS/waveform_collection/I06AU/WAVEFORM_I06AU_SAC'
output_folder = '/run/media/viblab/Markov11/Haykal/AnakKrakatauEWS/waveform_collection/I06AU/WAVEFORM_I06AU_CSV'
freqmin = 0.7 
freqmax = 4.0 
df_grouped_file_paths = sac_path_grouping(folder_path)

# Iterate over the DataFrame rows and process each file path
for index, row in df_grouped_file_paths.iterrows():
    file_path_pattern = row['GroupedFilePath']
    try:
        # Process each file path pattern with the sac_to_csv function
        csv_output_path = sac_to_csv(file_path_pattern, output_folder, freqmin, freqmax)
        print(f'CSV file created at: {csv_output_path}')
    except Exception as e:
        print(f"Error processing file pattern {file_path_pattern}: {e}")

### 4. Running Beamforming


In [ ]:
# 4.1 Running Beamforming on I06AU Waveform

base_folder_path = '/run/media/viblab/Markov11/Haykal/AnakKrakatauEWS/waveform_collection/I06AU/WAVEFORM_I06AU_SAC'
df_grouped_file_paths = sac_path_grouping(base_folder_path)

# Wrap the iterrows with tqdm to create a progress bar
for index, row in tqdm(df_grouped_file_paths.iterrows(), total=df_grouped_file_paths.shape[0], desc="Processing"):
    # Construct the full path to the grouped files
    full_grouped_path = f"{base_folder_path}/{row['GroupedFilePath']}"

    # Prepare the CLI command
    cli_command = f"infrapy run_fk --local-wvfrms \"{full_grouped_path}\" --"
    
    try:
        # Execute the CLI command
        subprocess.run(cli_command, shell=True, check=True)
        print(f"Command executed for: {full_grouped_path}")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while running the command for {full_grouped_path}: {e}")

In [ ]:
# 4.1 Running Beamforming on I52GB Waveform

base_folder_path = '/run/media/viblab/Markov11/Haykal/AnakKrakatauEWS/waveform_collection/I52GB/WAVEFORM_I52GB_SAC'
df_grouped_file_paths = sac_path_grouping(base_folder_path)

# Wrap the iterrows with tqdm to create a progress bar
for index, row in tqdm(df_grouped_file_paths.iterrows(), total=df_grouped_file_paths.shape[0], desc="Processing"):
    # Construct the full path to the grouped files
    full_grouped_path = f"{base_folder_path}/{row['GroupedFilePath']}"

    # Prepare the CLI command
    cli_command = f"infrapy run_fk --local-wvfrms \"{full_grouped_path}\" --"
    
    try:
        # Execute the CLI command
        subprocess.run(cli_command, shell=True, check=True)
        print(f"Command executed for: {full_grouped_path}")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while running the command for {full_grouped_path}: {e}")